In [9]:

#CBOW model version
import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

filename = 'text8.zip'

# Read the data into a list of strings.
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data

words = read_data(filename)

# convert words to lower cases
for item in words:
    item.lower()
        
print('Data size', len(words))




Data size 17005207


In [10]:
# Step 2: Build the dictionary and replace rare words with UNK token.
def build_dataset(words, min_cut_freq):
  count_org = [['UNK', -1]]
  count_org.extend(collections.Counter(words).most_common())
  count = [['UNK', -1]]
  for word, c in count_org:
    word_tuple = [word, c]
    if word == 'UNK': 
        count[0][1] = c
        continue
    if c > min_cut_freq:
        count.append(word_tuple)
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

min_cut_freq = 3 #cut off frequence smaller then 3 words
data, count, dictionary, reverse_dictionary = build_dataset(words, min_cut_freq)
vocabulary_size = len(reverse_dictionary)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:20], [reverse_dictionary[i] for i in data[:20]])
print('Vocab size: ', vocabulary_size)








Most common words (+UNK) [['UNK', 242339], ['the', 1061396], ['of', 593677], ['and', 416629], ['one', 411764]]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10572, 134, 1, 27350, 2, 1, 103] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english']
Vocab size:  82297


In [11]:
data_index = 0
def generate_cbow_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size, num_skips), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    batch_temp = np.ndarray(shape=(num_skips), dtype=np.int32)
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch_temp[j] = buffer[target]
    batch[i] = batch_temp
    labels[i,0] = buffer[skip_window]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels


In [52]:
# Step 4: Build and train a skip-gram model.
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

print(valid_examples)

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32,shape=[batch_size, skip_window * 2])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    # Embedding size is calculated as shape(train_inputs) + shape(embeddings)[1:]
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)
    print(embed.shape)
    reduced_embed = tf.div(tf.reduce_sum(embed, 1), skip_window*2)
    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  #print(nce_weights.dtype,nce_biases.dtype,reduced_embed.dtype,train_labels.dtype)
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights, biases=nce_biases, inputs=reduced_embed, labels=train_labels,
                     num_sampled=num_sampled, num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
  similarity = tf.matmul( valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.initialize_all_variables()

[28 65  0 87 22 99 62 13 63 20 51  5 97  6 44 72]
(128, 2, 128)


In [13]:
# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_cbow_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0
                
    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 100000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

tsne_plot = False # Change the false to true to invoke the tsne graph

Initialized
Average loss at step  0 :  304.979431152
Nearest to it: liquidity, gutman, spartan, greene, solms, artcyclopedia, authoritarianism, emi,
Nearest to years: lk, habibie, mutate, greenlanders, benefices, glorifying, hate, krag,
Nearest to states: bocce, galician, ynys, pogues, intramolecular, phagocytes, ont, communaute,
Nearest to used: specrate, staircases, iberville, cytology, sportsmedicine, worn, ffffff, nuclides,
Nearest to was: coffea, limnic, submission, angiotensin, subterranea, sutcliffe, nski, spim,
Nearest to into: formations, boatman, epi, sanader, throwers, horsetooth, tiara, steward,
Nearest to united: idlewild, lovers, ach, politican, emphasised, parasitism, blueness, gunfighter,
Nearest to up: rightist, astronautics, maxi, beira, reset, cantilever, gustav, demilitarisation,
Nearest to about: designator, lakeside, urges, facilitation, parameters, coogan, reinhardtii, embedding,
Nearest to would: pointing, aemilius, forebear, buddhanet, wortley, catapults, wellc

In [20]:
tsne_plot = True
# Step 6: Visualize the embeddings.
def plot_with_labels(low_dim_embs, labels, filename='tsne_CBOW.png'):
  assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
  plt.figure(figsize=(18, 18))  #in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i,:]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

try:
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  if(tsne_plot):
      tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
      plot_only = 300
      low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only,:])
      labels = [reverse_dictionary[i] for i in xrange(plot_only)]
      plot_with_labels(low_dim_embs, labels)

except ImportError:
  print("Please install sklearn and matplotlib to visualize embeddings.")

# Testing final embedding
input_dictionary = dict([(v,k) for (k,v) in reverse_dictionary.items()])

test_word_idx_a = input_dictionary.get('france') # replace france with other testing words such as king
test_word_idx_b = input_dictionary.get('paris')    # replace paris with other testing words such as man
test_word_idx_c = input_dictionary.get('rome')    # replace rome with other testing words such as woman

a = final_embeddings[test_word_idx_a,:]
b = final_embeddings[test_word_idx_b,:]
c = final_embeddings[test_word_idx_c,:]
ans = c + (b - a)

similarity = final_embeddings.dot(ans)

top_k = 4
nearest = (-similarity).argsort()[0:top_k+1]
print (nearest)
for k in xrange(top_k+1):
    close_word = reverse_dictionary[nearest[k]]
    print(close_word)


[  938  1055  2935  7231 24603]
rome
paris
orange
platinum
lemurs


In [51]:

d = final_embeddings[843,:]
test = final_embeddings.dot(d)

nearest = (-test).argsort()[0:10]
print (nearest)
for k in xrange(10):
    close_word = reverse_dictionary[nearest[k]]
    print(close_word)


[  843 18033 20847  4376 18799   874  5692 13704 19306  5469]
italy
cegep
bde
occasion
jumper
asia
focuses
andaman
marmoset
deliberately


In [30]:
print(input_dictionary.get('italy'))
input_dictionary.get('france')

843


303